In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("../input/feedback-prize-effectiveness/train.csv")
test = pd.read_csv("../input/feedback-prize-effectiveness/test.csv")

In [ ]:
train.head(10)

In [ ]:
test.head()

In [ ]:
train.dropna(how='any',inplace=True)
test.dropna(how='any',inplace=True)

In [ ]:
target_map = {"Adequate":1,"Effective":2,"Ineffective":0}
train["target"] = train["discourse_effectiveness"].map(target_map)
#test["target"] = test["discourse_effectiveness"].map(target_map)

In [ ]:
def convert_label(label):
    return '__label__'+str(label)

In [ ]:
train["target"] = train["target"].apply(convert_label)

In [ ]:
df_train = pd.DataFrame({'label': train["target"].tolist(),
                        'discourse_text': train['discourse_text'].tolist()})

In [ ]:
np.savetxt(r'ticket.train.txt', df_train.values, fmt='%s %s')

In [ ]:
! head ticket.train.txt

In [ ]:
import fasttext
! wc ticket.train.txt

In [ ]:
! head -n 45750 ticket.train.txt > tickets.train

In [ ]:
model = fasttext.train_supervised(input="tickets.train", lr=1.0, epoch=10, wordNgrams=2, bucket=500000, dim=100, loss='ova')

In [ ]:
model2 = fasttext.train_supervised(input="tickets.train", loss='ova')

In [ ]:
model.predict("it can decrease stress levels")

In [ ]:
pred_labels = model.predict(test['discourse_text'].apply(lambda x: str(x)).tolist(),k=3)

In [ ]:
pred_labels

In [ ]:
sample_submission = pd.read_csv('/kaggle/input/feedback-prize-effectiveness/sample_submission.csv')
sample_submission.head()

In [ ]:
sample_submission['discourse_id'] = test['discourse_id']
# sample_submission['Adequate'] = pred_labels[:,1]
# sample_submission['Effective'] = pred_labels[:,2]

In [ ]:
sample_submission.head()

In [ ]:
for i in range(10):
    sample_submission['Ineffective'][i]=pred_labels[1][i][2]

In [ ]:
for i in range(10):
    if pred_labels[0][i][0]=='__label__2':
        sample_submission['Effective'][i]=pred_labels[1][i][0]
    else:
        sample_submission['Effective'][i]=pred_labels[1][i][1]

In [ ]:
sample_submission

In [ ]:
for i in range(10):
    if pred_labels[0][i][0]=='__label__1':
        sample_submission['Adequate'][i]=pred_labels[1][i][0]
    else:
        sample_submission['Adequate'][i]=pred_labels[1][i][1]

In [ ]:
sample_submission

In [ ]:
sample_submission.to_csv("submission.csv",index=False)